In [1]:
import pandas as pd
from datetime import timedelta, date
import geopandas as gpd
from uuid import uuid4

Things to think about:
* what if control day not in temperature data set? (solution is to add more temperature data)
* what if control day is actually a heat wave?
* what if control day is close to a heat wave?

In [2]:
from datetime import datetime, timedelta
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
#import contextily as ctx
import matplotlib.pyplot as plt
datelist = pd.date_range(datetime.today(), periods=100).tolist()
from time import sleep
import dask.dataframe as dd
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import os
import glob

from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import geopandas as gpd



## Load heatwaves and get initial control dates

In [3]:
#heatwaves = pd.read_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwaves_90.csv')
heatwaves = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/turkey/THI_turkey_90_2022.csv')

In [4]:
heatwaves.month.unique()

array([8, 7, 9, 6, 5])

In [5]:
#may_hw = heatwaves.groupby('heat_ID').apply(lambda x: 5 in x['month'].unique()).reset_index()
#may_hw = may_hw[may_hw[0] == True]
#heatwaves = heatwaves[~heatwaves['heat_ID'].isin(may_hw['heat_ID'].values)]

In [6]:
heatwaves['month'].unique()

array([8, 7, 9, 6, 5])

In [7]:
heatwaves['datetime'] = pd.to_datetime(heatwaves['datetime'])

In [8]:
heatwaves['control_day'] = heatwaves['datetime'] - timedelta(days=7)


In [9]:
heatwaves['day_of_week'] = [x.weekday() for x in heatwaves['datetime']]

In [10]:
heatwaves_active = heatwaves[heatwaves['heatwave'] == 1]
heat_days = heatwaves_active.groupby('location')['datetime'].unique().reset_index().set_index('location')

In [11]:
heat_days = heat_days.to_dict()['datetime']

In [12]:
heatwaves_active.reset_index(inplace=True)

In [13]:
full_df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/daily_max/Turkey/THI_daily_max_turkey.csv')

full_df.rename(columns={'time': 'datetime'}, inplace=True)
full_df['datetime'] = pd.to_datetime(full_df['datetime'])
full_df['day'] = pd.DatetimeIndex(full_df['datetime']).day
full_df['month'] = pd.DatetimeIndex(full_df['datetime']).month
full_df['year'] = pd.DatetimeIndex(full_df['datetime']).year
full_df = full_df[~full_df['month'].isin([4, 10])]
full_df['location'] = full_df['adm2'] + ', ' + full_df['adm1']
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]

In [14]:
full_df['location'] = full_df['adm2'] + ', ' + full_df['adm1']

In [15]:
full_df.drop_duplicates(['location', 'datetime'], inplace=True)



In [16]:
full_df['temp_pct'] = full_df.groupby('location')['thi'].rank(pct=True)

In [17]:
full_df['temp_pct'].describe()

count    741285.000000
mean          0.500654
std           0.288675
min           0.001307
25%           0.250980
50%           0.500654
75%           0.750327
max           1.000000
Name: temp_pct, dtype: float64

In [18]:
heatwaves_active['num_iters'] = 'test'

In [19]:

for index, row in heatwaves_active.iterrows():
    subset = full_df[full_df['location'] == row['location']]
    control_day = heatwaves_active.iloc[index, -3]
    i = 0
    try:
        while ((subset[subset['datetime'] == control_day]['temp_pct'].values[0]>= 0.8) | 
               (subset[(subset['datetime'] == control_day)]['temp_pct'].values[0]<= 0.2)):


            heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] - timedelta(days=7)
            control_day = heatwaves_active.iloc[index, -3]

    except:
        heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
        control_day = heatwaves_active.iloc[index, -3]

        while ((subset[subset['datetime'] == control_day]['temp_pct'].values[0]>= 0.8) | 
               (subset[(subset['datetime'] == control_day)]['temp_pct'].values[0]<= 0.2)):
        
            heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
            control_day = heatwaves_active.iloc[index, -3]
            
    print(index / len(heatwaves_active))
            


0.0
6.729927989770509e-05
0.00013459855979541018
0.0002018978396931153
0.00026919711959082036
0.00033649639948852547
0.0004037956793862306
0.0004710949592839357
0.0005383942391816407
0.0006056935190793458
0.0006729927989770509
0.000740292078874756
0.0008075913587724611
0.0008748906386701663
0.0009421899185678714
0.0010094891984655764
0.0010767884783632815
0.0011440877582609866
0.0012113870381586917
0.0012786863180563968
0.0013459855979541019
0.001413284877851807
0.001480584157749512
0.0015478834376472172
0.0016151827175449223
0.0016824819974426274
0.0017497812773403325
0.0018170805572380376
0.0018843798371357427
0.0019516791170334478
0.0020189783969311527
0.002086277676828858
0.002153576956726563
0.002220876236624268
0.002288175516521973
0.0023554747964196782
0.0024227740763173833
0.0024900733562150884
0.0025573726361127935
0.0026246719160104987
0.0026919711959082038
0.002759270475805909
0.002826569755703614
0.002893869035601319
0.002961168315499024
0.0030284675953967293
0.003095766875

In [20]:
heatwaves_active['thi'].min()

20.65739666974107

In [21]:
heatwaves['pct'].min()

0.9006535947712418

In [22]:
#export.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave_control/heatwave_control_90_v2.csv')
heatwaves_active.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/paper2/turkey/THI_heatwave_control_90.csv')